## Deliverable 2. Create a Customer Travel Destinations Map.

In [60]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import google_maps_key

# Configure gmaps API key
gmaps.configure(api_key=google_maps_key)

In [62]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Philippe,RE,-21.3585,55.7679,76.59,81,100,6.22,overcast clouds
1,1,Yellowknife,CA,62.4560,-114.3525,12.42,99,75,5.75,light snow
2,2,Roanoke Rapids,US,36.4615,-77.6542,53.24,62,75,17.27,broken clouds
3,3,Rikitea,PF,-23.1203,-134.9692,77.92,77,5,21.92,clear sky
4,4,Geraldton,AU,-28.7667,114.6000,68.41,83,100,10.36,overcast clouds


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Enter minimum temperature:"))
max_temp = float(input("Enter maximum temperature:"))

Enter minimum temperature:70
Enter maximum temperature:95


In [35]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & (city_data_df["Max Temp"]<= max_temp)]
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279 entries, 0 to 687
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              279 non-null    int64  
 1   City                 279 non-null    object 
 2   Country              277 non-null    object 
 3   Lat                  279 non-null    float64
 4   Lng                  279 non-null    float64
 5   Max Temp             279 non-null    float64
 6   Humidity             279 non-null    int64  
 7   Cloudiness           279 non-null    int64  
 8   Wind Speed           279 non-null    float64
 9   Current Description  279 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 24.0+ KB


In [38]:
# 4a. Determine if there are any empty rows.
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279 entries, 0 to 687
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              279 non-null    int64  
 1   City                 279 non-null    object 
 2   Country              277 non-null    object 
 3   Lat                  279 non-null    float64
 4   Lng                  279 non-null    float64
 5   Max Temp             279 non-null    float64
 6   Humidity             279 non-null    int64  
 7   Cloudiness           279 non-null    int64  
 8   Wind Speed           279 non-null    float64
 9   Current Description  279 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 24.0+ KB


In [39]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filtered_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277 entries, 0 to 687
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              277 non-null    int64  
 1   City                 277 non-null    object 
 2   Country              277 non-null    object 
 3   Lat                  277 non-null    float64
 4   Lng                  277 non-null    float64
 5   Max Temp             277 non-null    float64
 6   Humidity             277 non-null    int64  
 7   Cloudiness           277 non-null    int64  
 8   Wind Speed           277 non-null    float64
 9   Current Description  277 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 23.8+ KB


In [40]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Saint-Philippe,RE,76.59,overcast clouds,-21.3585,55.7679,
3,Rikitea,PF,77.92,clear sky,-23.1203,-134.9692,
6,Butaritari,KI,81.63,light rain,3.0707,172.7902,
9,Vila Velha,BR,82.35,clear sky,-20.3297,-40.2925,
11,Hermanus,ZA,75.90,overcast clouds,-34.4187,19.2345,
18,Tiarei,PF,75.22,light rain,-17.5333,-149.3333,
21,Hilo,US,78.53,broken clouds,19.7297,-155.0900,
23,Namatanai,PG,79.66,overcast clouds,-3.6667,152.4333,
24,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,
29,Bambous Virieux,MU,81.21,light rain,-20.3428,57.7575,


In [49]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_maps_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    city_lat = row['Lat']
    city_lng = row['Lng']
    params['location'] = str(city_lat) + ',' + str(city_lng)
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    print(f"Retrieving Results for Hotel {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        print(f"Closest Hotel in {row['City']} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, 'Hotel Name'] = np.nan
        

Retrieving Results for Hotel 0: Saint-Philippe.
Closest Hotel in Saint-Philippe is Chambres d'hôte "La Trinité".
Retrieving Results for Hotel 3: Rikitea.
Closest Hotel in Rikitea is People ThankYou.
Retrieving Results for Hotel 6: Butaritari.
Closest Hotel in Butaritari is Isles Sunset Lodge.
Retrieving Results for Hotel 9: Vila Velha.
Closest Hotel in Vila Velha is Quality Suites Vila Velha.
Retrieving Results for Hotel 11: Hermanus.
Closest Hotel in Hermanus is Misty Waves Boutique Hotel.
Retrieving Results for Hotel 18: Tiarei.
Closest Hotel in Tiarei is Le Rocher de Tahiti.
Retrieving Results for Hotel 21: Hilo.
Closest Hotel in Hilo is Hilo Hawaiian Hotel.
Retrieving Results for Hotel 23: Namatanai.
Closest Hotel in Namatanai is Seagulls Inn Namatani.
Retrieving Results for Hotel 24: Vaini.
Closest Hotel in Vaini is Keleti Beach Resort.
Retrieving Results for Hotel 29: Bambous Virieux.
Closest Hotel in Bambous Virieux is Casa Tia Villa.
Retrieving Results for Hotel 30: Mahebourg.


Closest Hotel in Bathsheba is Atlantis Hotel.
Retrieving Results for Hotel 218: Ternate.
Closest Hotel in Ternate is Sahid Bela Ternate Hotel.
Retrieving Results for Hotel 219: Kudahuvadhoo.
Closest Hotel in Kudahuvadhoo is Niyama Private Islands Maldives.
Retrieving Results for Hotel 220: Bandarbeyla.
Closest Hotel in Bandarbeyla is JABIR HOTEL.
Retrieving Results for Hotel 221: Caravelas.
Closest Hotel in Caravelas is Pousada dos Navegantes.
Retrieving Results for Hotel 222: Sola.
Closest Hotel in Sola is Leumerus Bungalows.
Retrieving Results for Hotel 224: Ulladulla.
Closest Hotel in Ulladulla is Sandpiper Motel.
Retrieving Results for Hotel 225: Mahajanga.
Closest Hotel in Mahajanga is Restaurant Greedy Coco Lodge Majunga.
Retrieving Results for Hotel 226: Tupi Paulista.
Closest Hotel in Tupi Paulista is Hotel São Bento.
Retrieving Results for Hotel 227: Karratha.
Closest Hotel in Karratha is ibis Styles Karratha.
Retrieving Results for Hotel 228: Nhulunbuy.
Closest Hotel in Nhulu

Closest Hotel in Souillac is Shanti Maurice Resort & Spa.
Retrieving Results for Hotel 416: Key Largo.
Closest Hotel in Key Largo is Bayside Inn Key Largo.
Retrieving Results for Hotel 418: Kouango.
Missing field/result... skipping.
Retrieving Results for Hotel 419: Takoradi.
Closest Hotel in Takoradi is Raybow International Hotel.
Retrieving Results for Hotel 422: Mackay.
Closest Hotel in Mackay is International Lodge Motel.
Retrieving Results for Hotel 423: Bluefields.
Closest Hotel in Bluefields is Nicaraguan Typical House Downtown.
Retrieving Results for Hotel 426: Mogadishu.
Closest Hotel in Mogadishu is Hotel Juba.
Retrieving Results for Hotel 427: Tomatlan.
Closest Hotel in Tomatlan is El Salvial.
Retrieving Results for Hotel 428: Kavieng.
Closest Hotel in Kavieng is Nusa Island Retreat.
Retrieving Results for Hotel 429: Sao Filipe.
Closest Hotel in Sao Filipe is Hotel Xaguate.
Retrieving Results for Hotel 430: Kailua.
Closest Hotel in Kailua is Sheffield House Bed & Breakfast.


Closest Hotel in San Lorenzo is Hotel del Sur.
Retrieving Results for Hotel 638: Yulara.
Closest Hotel in Yulara is Desert Gardens Hotel - Ayers Rock Resort.
Retrieving Results for Hotel 641: Kikwit.
Closest Hotel in Kikwit is HOTEL PLACE YALABA.
Retrieving Results for Hotel 644: Rawson.
Closest Hotel in Rawson is Hosteria Sampedro.
Retrieving Results for Hotel 645: Taveta.
Closest Hotel in Taveta is Challa Hotel.
Retrieving Results for Hotel 649: Ratholo.
Missing field/result... skipping.
Retrieving Results for Hotel 658: Huarmey.
Closest Hotel in Huarmey is Hotel Miramar.
Retrieving Results for Hotel 659: Jibuti.
Closest Hotel in Jibuti is Orchid Main Camp Goubet.
Retrieving Results for Hotel 665: Nabulao.
Closest Hotel in Nabulao is Melrose Beach Resort.
Retrieving Results for Hotel 667: Daru.
Closest Hotel in Daru is Daru Lodge.
Retrieving Results for Hotel 670: Molepolole.
Closest Hotel in Molepolole is Butterfly Tree Guesthouse.
Retrieving Results for Hotel 672: Anito.
Closest Ho

In [52]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Saint-Philippe,RE,76.59,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
3,Rikitea,PF,77.92,clear sky,-23.1203,-134.9692,People ThankYou
6,Butaritari,KI,81.63,light rain,3.0707,172.7902,Isles Sunset Lodge
9,Vila Velha,BR,82.35,clear sky,-20.3297,-40.2925,Quality Suites Vila Velha
11,Hermanus,ZA,75.90,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
18,Tiarei,PF,75.22,light rain,-17.5333,-149.3333,Le Rocher de Tahiti
21,Hilo,US,78.53,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
23,Namatanai,PG,79.66,overcast clouds,-3.6667,152.4333,Seagulls Inn Namatani
24,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
29,Bambous Virieux,MU,81.21,light rain,-20.3428,57.7575,Casa Tia Villa


In [53]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [58]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))